# Inspect script and extract functions and methods

In [2]:
# https://docs.python.org/3/library/inspect.html#inspect.getdoc

import inspect

In [5]:
sig.from_function

<bound method Signature.from_function of <class 'inspect.Signature'>>

In [7]:
inspect.getmodulename('example_module.py')
#inspect.getdoc(object)

'example_module'

# Create pytest skeleton

In [38]:
import example_module
methods = inspect.getmembers(example_module.Arithmetic, predicate=inspect.isfunction) # predicate=inspect.ismethod
with open("test_example_module.py","w") as f:
    for method in methods:
        method_name = method[0]
        method_signature = inspect.signature(method[1])
        args = [arg for arg in method_signature.parameters.keys() if arg != 'self']
        if args:
            skeleton = (
f"""
class TestClass{method[0].capitalize()}():

    @pytest.mark.parametrize(
        "params,expected",
        [
            [
                {{
                    '{args[0]}': [0, 1, 2],
                    '{args[1]}': [-1, -2, 3]
                }},
                [expected]
                )
            ]
        ]
    )
    def test_return(self, params, expected):
        assert example_module.Arithmetic.addition(params['{args[0]}'], params['{args[1]}']).equals(expected)
            """
)
            f.write(skeleton)